# Dicom seg test

Voglio aggiungere al container il check sul json che viene creato e un check sul file dicom

In [6]:
import pydicom
import os

In [22]:
def checkSlicesDicom(directory):
    """
    Checks for the presence and continuity of DICOM slices in a given directory.

    This function iterates over all files in the specified directory, attempts to read
    them as DICOM files, and extracts the 'InstanceNumber' (or 'SliceLocation' if 
    'InstanceNumber' is not available) for each file. It then checks whether the slice
    numbers are consecutive and reports any missing slices.

    Parameters:
    -----------
    directory : str
        The path to the directory containing DICOM files to be checked.

    Returns:
    --------
    None
        Prints a message indicating whether all slices are present and consecutive.
        If slices are missing, the function prints a list of the missing slice numbers.

    Raises:
    -------
    Exception:
        If any file cannot be read as a DICOM file, an error message will be printed
        indicating which file caused the issue.
    """
    slices = []

    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        try:
            ds = pydicom.dcmread(file_path)
            if hasattr(ds, 'InstanceNumber'):
                slices.append(ds.InstanceNumber)
            elif hasattr(ds, 'SliceLocation'):
                slices.append(ds.SliceLocation)
        except Exception as e:
            print(f"Error reading  {file_name}: {e}")
    slices.sort()
    missing_slices = []
    for i in range(slices[0], slices[-1] + 1):
        if i not in slices:
            missing_slices.append(i)

    if missing_slices:
        print(f"Miss the following slices: {missing_slices}")
    else:
        print("All slices exist and are consecutive")


In [23]:
checkSlicesDicom(path)

All slices exist and are consecutive


---
Check on json file

Sistemo il controllo sul json in modo che sia più stringente e performi meglio 

In [31]:
import json
import os


In [32]:
def update_segment_algorithm_name(file_path):
    """
    Updates the 'SegmentAlgorithmName' field in a JSON file for non-manual segments.

    This function reads a JSON file, iterates through the 'segmentAttributes' list, 
    and checks each segment's 'SegmentAlgorithmType'. If the type is not 'MANUAL' 
    and the 'SegmentAlgorithmName' field is missing, it prompts the user to input 
    a value for 'SegmentAlgorithmName' for the corresponding segment.

    After updating the segments, the modified JSON is saved back to the same file.

    Parameters:
    -----------
    file_path : str
        The path to the JSON file to be updated.

    Returns:
    --------
    None
        The function directly updates the JSON file and does not return any value.

    Notes:
    ------
    - The function assumes that the 'segmentAttributes' field is a list of lists, 
      where each inner list contains dictionaries representing segments.
    - The user is prompted to input the 'SegmentAlgorithmName' value for each segment 
      that is not 'MANUAL' and lacks a 'SegmentAlgorithmName'.
    - The updated JSON file is written back with the same file name.
    """

    with open(file_path, 'r') as file:
        data = json.load(file)

    for segment_list in data['segmentAttributes']:
        for segment in segment_list:
            if segment.get('SegmentAlgorithmType') != 'MANUAL':
                if 'SegmentAlgorithmName' not in segment:
                    segment['SegmentAlgorithmName'] = input(f"Enter the SegmentAlgorithmName for the label ID {segment['labelID']}: ")

    with open(file_path, 'w') as file:
        json.dump(data, file, indent=2)


In [34]:
file_path = "DATA/EucaimShared/Test2/interobs05/dicomnifti/seg/meta.json"
update_segment_algorithm_name(file_path)

In [35]:

MANDATORY_FIELDS = {
    'SegmentDescription': None,
    'SegmentAlgorithmType': None,
    'SegmentedPropertyCategoryCodeSequence': ['CodeValue', 'CodingSchemeDesignator', 'CodeMeaning'],
    'SegmentedPropertyTypeCodeSequence': ['CodeValue', 'CodingSchemeDesignator', 'CodeMeaning'],
    'labelID': None
}

CONDITIONAL_FIELDS = {
    'SegmentAlgorithmType': {
        'conditions': lambda value: value != 'MANUAL',
        'required_fields': ['SegmentAlgorithmName']
    },
    'SegmentType': {
        'conditions': lambda value: value == 'TISSUE',
        'required_fields': ['SegmentedPropertyTypeCodeSequence']
    },
    'SegmentedPropertyCategoryCodeSequence': {
        'conditions': lambda value: isinstance(value, list) and len(value) > 0,
        'required_subfields': ['CodeValue', 'CodingSchemeDesignator', 'CodeMeaning']
    },
    'SegmentedPropertyTypeCodeSequence': {
        'conditions': lambda value: isinstance(value, list) and len(value) > 0,
        'required_subfields': ['CodeValue', 'CodingSchemeDesignator', 'CodeMeaning']
    }
}

def check_mandatory_fields(json_data):
    """
    Verifica che tutti i campi obbligatori, inclusi eventuali sottocampi, siano presenti in ciascun segmento.
    Gestisce anche la logica condizionale per i campi basata sui valori di altri campi.

    Parameters:
    -----------
    json_data : dict
        Il contenuto del file JSON già caricato.

    Returns:
    --------
    bool
        True se tutti i campi obbligatori sono presenti, False altrimenti.
    
    missing_fields : dict
        Un dizionario che elenca i campi e sottocampi mancanti per ciascun segmento.
    """
    missing_fields = {}

    # Itera attraverso tutti i segmenti nel file JSON
    for i, segment_list in enumerate(json_data.get('segmentAttributes', [])):
        for j, segment in enumerate(segment_list):
            segment_missing = []

            # Verifica i campi obbligatori e i sottocampi
            for field, subfields in MANDATORY_FIELDS.items():
                if field not in segment:
                    segment_missing.append(field)
                elif subfields:  # Se ci sono sottocampi obbligatori
                    if isinstance(segment[field], list) and len(segment[field]) > 0:
                        sequence_item = segment[field][0]  # Considera il primo elemento della sequenza
                        missing_subfields = [subfield for subfield in subfields if subfield not in sequence_item]
                        if missing_subfields:
                            segment_missing.append(f"{field}: missing {missing_subfields}")
                    else:
                        segment_missing.append(f"{field}: expected a non-empty sequence")
            
            # Verifica i campi condizionali complessi
            for field, condition_info in CONDITIONAL_FIELDS.items():
                if field in segment:
                    # Se la condizione è soddisfatta, verifica i campi richiesti
                    condition_met = condition_info['conditions'](segment[field])
                    if condition_met:
                        if 'required_fields' in condition_info:
                            for required_field in condition_info['required_fields']:
                                if required_field not in segment:
                                    segment_missing.append(f"Missing conditional field {required_field} due to {field} value")
                        # Verifica sottocampi condizionali, se presenti
                        if 'required_subfields' in condition_info:
                            if isinstance(segment[field], list) and len(segment[field]) > 0:
                                sequence_item = segment[field][0]  # Controlla il primo elemento della sequenza
                                for subfield in condition_info['required_subfields']:
                                    if subfield not in sequence_item:
                                        segment_missing.append(f"{field}: missing subfield {subfield}")
                            else:
                                segment_missing.append(f"{field}: expected a non-empty sequence")

            if segment_missing:
                missing_fields[f"Segment {i + 1} (LabelID {segment.get('labelID', 'Unknown')})"] = segment_missing

    return len(missing_fields) == 0, missing_fields


In [36]:
with open(file_path, 'r') as file:
    data = json.load(file)

all_fields_present, missing_fields = check_mandatory_fields(data)

if all_fields_present:
    print(f"Il file {file_path} contiene tutti i campi obbligatori.")
else:
    print(f"Il file {file_path} è mancante dei seguenti campi:")
    for segment, fields in missing_fields.items():
        print(f"  - {segment}: {fields}")

Il file DATA/EucaimShared/Test2/interobs05/dicomnifti/seg/meta.json è mancante dei seguenti campi:
  - Segment 1 (LabelID 1): ['SegmentedPropertyCategoryCodeSequence: expected a non-empty sequence', 'SegmentedPropertyTypeCodeSequence: expected a non-empty sequence']
  - Segment 2 (LabelID 2): ['SegmentedPropertyCategoryCodeSequence: expected a non-empty sequence', 'SegmentedPropertyTypeCodeSequence: expected a non-empty sequence']
  - Segment 3 (LabelID 3): ['SegmentedPropertyCategoryCodeSequence: expected a non-empty sequence', 'SegmentedPropertyTypeCodeSequence: expected a non-empty sequence']
  - Segment 4 (LabelID 4): ['SegmentedPropertyCategoryCodeSequence: expected a non-empty sequence', 'SegmentedPropertyTypeCodeSequence: expected a non-empty sequence']
  - Segment 5 (LabelID 5): ['SegmentedPropertyCategoryCodeSequence: expected a non-empty sequence', 'SegmentedPropertyTypeCodeSequence: expected a non-empty sequence']
  - Segment 6 (LabelID 6): ['SegmentedPropertyCategoryCodeSeq

Il file DATA/EucaimShared/Test1/LUNG1-001/dicomnifti/seg/meta.json è mancante dei seguenti campi:
  - Segment 1 (LabelID 1): ['SegmentedPropertyCategoryCodeSequence: expected a non-empty sequence', 'SegmentedPropertyTypeCodeSequence: expected a non-empty sequence']
  - Segment 2 (LabelID 2): ['SegmentedPropertyCategoryCodeSequence: expected a non-empty sequence', 'SegmentedPropertyTypeCodeSequence: expected a non-empty sequence']
  - Segment 3 (LabelID 3): ['SegmentedPropertyCategoryCodeSequence: expected a non-empty sequence', 'SegmentedPropertyTypeCodeSequence: expected a non-empty sequence']
  - Segment 4 (LabelID 4): ['SegmentedPropertyCategoryCodeSequence: expected a non-empty sequence', 'SegmentedPropertyTypeCodeSequence: expected a non-empty sequence']
